### Install snowpark libraries, Connect to Snowflake DB, Load dummy data

In [100]:
!pip install snowflake-snowpark-python

In [80]:
import pandas as pd
from datetime import datetime

date_df = pd.DataFrame(columns=['user_id','timestamp'])
date_df['user_id']=range(0,245,1)
date_df['timestamp'] = pd.date_range(datetime.today(), periods=245).tolist()

lat_long_file = 'world_country_and_usa_states_latitude_and_longitude_values.csv'
df = pd.concat([date_df,pd.read_csv(lat_long_file)[['latitude','longitude']]],axis=1)
df.rename(columns={'user_id':'USER_ID','timestamp':'TIMESTAMP','latitude':'LAT','longitude':'LONG'},inplace=True)
df.head()

,USER_ID,TIMESTAMP,LAT,LONG
0,0,2024-08-08 22:17:29.566880,42.546245,1.601554
1,1,2024-08-09 22:17:29.566880,23.424076,53.847818
2,2,2024-08-10 22:17:29.566880,33.939110,67.709953
3,3,2024-08-11 22:17:29.566880,17.060816,-61.796428
4,4,2024-08-12 22:17:29.566880,18.220554,-63.068615


In [ ]:
from snowflake.snowpark import Session, version
import json

# Create Snowflake Session object
connection_parameters = json.load(open('cred.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))

In [93]:
session.create_dataframe(df).write.save_as_table(table_name='transactions', mode='overwrite')

### Install tecton Libs, Connect to workspace, create batch datasource and create featureview and apply

In [ ]:
!pip install 'tecton[rift,snowflake]==0.10.0b7'

In [2]:
!tecton login https://demo-apollo.tecton.ai/

Requesting authorization for Tecton CLI via browser. 
If browser doesn't open automatically, use `tecton login --manual`
✅ Authentication successful!
✅ Updated configuration at /Users/knadadur/.tecton/config


In [11]:
!tecton init

Local feature repository root set to /Users/knadadur/Documents/tecton-coinbase

💡 We recommend tracking these files in git:
     /Users/knadadur/Documents/tecton-coinbase/.tecton
     /Users/knadadur/Documents/tecton-coinbase/repo.yaml

💡 Run `tecton apply` to apply the feature repository to the Tecton cluster.


In [94]:
from tecton import BatchSource, SnowflakeConfig, Entity, batch_feature_view, FeatureService
from datetime import datetime, timedelta

In [101]:
transactions_batch = BatchSource(
    name="transactions",
    batch_config=SnowflakeConfig(
        url="https://tnb84480.snowflakecomputing.com//",
        database="transactions_db",
        schema="transaction_schema",
        warehouse="xs_wh",
        table="transactions",
        timestamp_field = "TIMESTAMP",
        user=<snowflake-user-id>,
        password=<password>,
    )
)

In [102]:
user = Entity(name="user", join_keys=["USER_ID"])

In [103]:
from tecton import batch_feature_view


@batch_feature_view(
    sources=[transactions_batch], 
    entities=[user], 
    mode="snowflake_sql",
    online=True,
    offline=True,
    batch_schedule=timedelta(days=1),
)
def transaction_location(transactions):
    return f"""
        SELECT
            USER_ID,
            TIMESTAMP,
            LAT,
            LONG
        FROM
            {transactions}
        """

In [104]:
fraud_detection_service = FeatureService(
    name='fraud_detection_service',
    features=[
        transaction_location
    ],
)

In [105]:
!tecton apply

Using workspace "coinbase-test" on cluster https://demo-apollo.tecton.ai
✅ Imported 0 Python modules from the feature repository
✅ Imported 0 Python modules from the feature repository
⚠️  Running Tests: No tests found.
✅ Collecting local feature declarations
✅ Performing server-side feature validation: Initializing.
🎉 The remote and local state are the same, nothing to do!


In [106]:
import tecton

ws = tecton.get_workspace("coinbase-test")

In [108]:
data_source = ws.get_data_source("transactions_batch")

TectonValidationError: There are no commits to restore from in this workspace, trace ID: 0774a08c4c53a9ffe134b50bdc8ba583

In [ ]:
git revert <commit_hash>